# Idaho -- City Sustainability

In [1]:
%pip install census us

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib as plt

from census import Census
from us import states

import plotly.graph_objects as go

In [3]:
c = Census('fb97753783c42ae57fe1a640e38fe04e921e5d1a')

## Get's the 5 largest cities in Idaho

In [17]:
keys = ['NAME' ,'P002001','P002002','P002005','H001001','P013001','H003001','P027001','H005001','H005002','H005003','H005004','H005005','H005006','H005007']

renames_2000 = {
        'NAME' : 'City_Name',
        'place': 'FIPS',
        'P002001': 'Total_Population_2000',
        'P002002':'Total_Urban_Population_2000',
        'P002005':'Total_Rural_Population_2000',
        'H001001': 'Total_Housing_2000',
        'P013001': 'Median_Age_2000',
        'H003001': 'Occupancy_Status_For_Housing_Units_2000',
        'P027001': 'Presence_of_Non-Relatives_2000',
        'H005001': 'Vacancy_Status_2000',
        'H005002': 'For_Rent_2000',
        'H005003': 'Rented_Not_Occupied_2000',
        'H005004': 'For_Sale_Only_2000',
        'H005005': 'Sold_Not_Occupied_2000',
        'H005006': 'For_Seasonal_Recreational_Or_Occasional_Use_2000',
        'H005007': 'For_Migrant_Workers_2000'
}

renames_2010 = {
        'NAME' : 'City_Name',
        'place': 'FIPS',
        'P002001': 'Total_Population_2010',
        'P002002':'Total_Urban_Population_2010',
        'P002005':'Total_Rural_Population_2010',
        'H001001': 'Total_Housing_2010',
        'P013001': 'Median_Age_2010',
        'H003001': 'Occupancy_Status_For_Housing_Units_2010',
        'P027001': 'Presence_of_Non-Relatives_2010',
        'H005001': 'Vacancy_Status_2010',
        'H005002': 'For_Rent_2010',
        'H005003': 'Rented_Not_Occupied_2010',
        'H005004': 'For_Sale_Only_2010',
        'H005005': 'Sold_Not_Occupied_2010',
        'H005006': 'For_Seasonal_Recreational_Or_Occasional_Use_2010',
        'H005007': 'For_Migrant_Workers_2010'}

city_2010 = c.sf1.state_place(keys, states.ID.fips, '*', year=2010)
c_pop_2010 = pd.DataFrame.from_records(city_2010)
c_pop_2010_50000 = c_pop_2010.rename(columns=renames_2010)

In [18]:
c_pop_2010_50000.head()

,Total_Housing_2010,Occupancy_Status_For_Housing_Units_2010,Vacancy_Status_2010,For_Rent_2010,Rented_Not_Occupied_2010,For_Sale_Only_2010,Sold_Not_Occupied_2010,For_Seasonal_Recreational_Or_Occasional_Use_2010,For_Migrant_Workers_2010,City_Name,Total_Population_2010,Total_Urban_Population_2010,Total_Rural_Population_2010,Median_Age_2010,Presence_of_Non-Relatives_2010,FIPS,state
0,667.0,667.0,52.0,4.0,1.0,14.0,2.0,6.0,0.0,"Aberdeen city, Idaho",1994.0,0.0,1994.0,28.1,615.0,00100,16
1,45.0,45.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,"Acequia city, Idaho",124.0,0.0,124.0,36.0,43.0,00280,16
2,138.0,138.0,25.0,6.0,1.0,4.0,3.0,5.0,1.0,"Albion city, Idaho",267.0,0.0,267.0,42.8,113.0,01000,16
3,1612.0,1612.0,138.0,45.0,4.0,32.0,5.0,6.0,2.0,"American Falls city, Idaho",4457.0,4450.0,7.0,30.2,1474.0,01900,16
4,4747.0,4747.0,271.0,87.0,7.0,74.0,9.0,37.0,1.0,"Ammon city, Idaho",13816.0,13693.0,123.0,29.6,4476.0,01990,16


In [19]:
city_2000 = c.sf1.state_place(keys, states.ID.fips, '*', year=2000)
c_pop_2000 = pd.DataFrame.from_records(city_2000)
c_pop_2000_50000 = c_pop_2000.rename(columns=renames_2000)

In [20]:
c_pop_2000_50000.drop(columns=['City_Name', 'state'], inplace=True)

In [21]:
c_pop_2000_50000.head()

,Total_Housing_2000,Occupancy_Status_For_Housing_Units_2000,Vacancy_Status_2000,For_Rent_2000,Rented_Not_Occupied_2000,For_Sale_Only_2000,Sold_Not_Occupied_2000,For_Seasonal_Recreational_Or_Occasional_Use_2000,For_Migrant_Workers_2000,Total_Population_2000,Total_Urban_Population_2000,Total_Rural_Population_2000,Median_Age_2000,Presence_of_Non-Relatives_2000,FIPS
0,1088,1088,119,22,17,2,9,1,68,3193,0,0,27.9,3193,28360
1,20627,20627,1293,556,298,87,84,1,267,51466,0,0,28.8,51466,64090
2,77850,77850,3412,1463,739,302,353,1,554,185787,0,0,32.8,185787,08830
3,9603,9603,640,268,152,40,23,1,156,25967,0,0,28.8,25967,12250
4,4048,4048,184,28,81,18,15,0,42,11085,0,0,35.2,11085,23410


In [22]:
c_pop_2000_50000.set_index('FIPS', inplace=True)
c_pop_2010_50000.set_index('FIPS', inplace=True)

In [23]:
id_join = c_pop_2000_50000.join(c_pop_2010_50000, on='FIPS')

In [24]:
id_join.head()

,Total_Housing_2000,Occupancy_Status_For_Housing_Units_2000,Vacancy_Status_2000,For_Rent_2000,Rented_Not_Occupied_2000,For_Sale_Only_2000,Sold_Not_Occupied_2000,For_Seasonal_Recreational_Or_Occasional_Use_2000,For_Migrant_Workers_2000,Total_Population_2000,...,Sold_Not_Occupied_2010,For_Seasonal_Recreational_Or_Occasional_Use_2010,For_Migrant_Workers_2010,City_Name,Total_Population_2010,Total_Urban_Population_2010,Total_Rural_Population_2010,Median_Age_2010,Presence_of_Non-Relatives_2010,state
FIPS,,,,,,,,,,,,,,,,,,,,,
28360,1088,1088,119,22,17,2,9,1,68,3193,...,5.0,11.0,0.0,"Fort Hall CDP, Idaho",3201.0,0.0,3201.0,31.2,1014.0,16
64090,20627,20627,1293,556,298,87,84,1,267,51466,...,70.0,97.0,3.0,"Pocatello city, Idaho",54255.0,54189.0,66.0,30.2,20832.0,16
08830,77850,77850,3412,1463,739,302,353,1,554,185787,...,234.0,595.0,3.0,"Boise City city, Idaho",205671.0,204776.0,895.0,35.3,85704.0,16
12250,9603,9603,640,268,152,40,23,1,156,25967,...,81.0,39.0,0.0,"Caldwell city, Idaho",46237.0,46118.0,119.0,28.2,14895.0,16
23410,4048,4048,184,28,81,18,15,0,42,11085,...,42.0,93.0,0.0,"Eagle city, Idaho",19908.0,19511.0,397.0,40.6,7069.0,16


In [25]:
id_join['Total_Population_2000'] = id_join['Total_Population_2000'].astype('i8')

In [26]:
id_join =  id_join.nlargest(5, 'Total_Population_2000')

In [27]:
fig = go.Figure(data=[
    go.Bar(name='2000_pop', x=id_join['City_Name'], y=id_join['Total_Population_2000']),
    go.Bar(name='2010_pop', x=id_join['City_Name'], y=id_join['Total_Population_2010']),
    go.Bar(name='2000_housing', x=id_join['City_Name'], y=id_join['Total_Housing_2000']),
    go.Bar(name='2010_housing', x=id_join['City_Name'], y=id_join['Total_Housing_2010']),
    go.Bar(name='2000_non-relatives', x=id_join['City_Name'], y=id_join['Presence_of_Non-Relatives_2000']),
    go.Bar(name='2010_non-relatives', x=id_join['City_Name'], y=id_join['Presence_of_Non-Relatives_2010']),
])
fig.update_layout(barmode='group')
fig.show()

## American Community Servey

In [28]:
acs_years = {}
for x in range(2012, 2018):
    acs_test = c.acs5.state_place(('NAME',
'B01003_001E',
'B00002_001E',
'B09018_007E'), states.ID.fips, '*', year=x)
    acs_years[x] = pd.DataFrame.from_records(acs_test)
    print(x)
    acs_years[x] = acs_years[x].rename(columns={
        'NAME' : 'City_Name',
        'place': 'FIPS',
        'B01003_001E': 'Total_Population_{}'.format(x),
        'B00002_001E': 'Total_Housing_{}'.format(x),
        'B09018_007E': 'Presence_of_Non-Relatives_{}'.format(x)})

2012
2013
2014
2015
2016
2017


In [29]:
acs_years[2013].head()

,Total_Housing_2013,Total_Population_2013,Presence_of_Non-Relatives_2013,City_Name,FIPS,state
0,121.0,2318.0,20.0,"Aberdeen city, Idaho",00100,16
1,22.0,115.0,0.0,"Acequia city, Idaho",00280,16
2,55.0,309.0,0.0,"Albion city, Idaho",01000,16
3,87.0,4405.0,9.0,"American Falls city, Idaho",01900,16
4,357.0,13993.0,0.0,"Ammon city, Idaho",01990,16
